In [2]:
# !wget https://github.com/alexeygrigorev/minsearch/blob/main/minsearch.py

# !uv pip install minsearch

In [16]:
from minsearch import Index
import json

from groq import Groq
import os
import openai

In [21]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

# docs_raw

In [5]:
documets = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documets.append(doc)

documets[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
# Create and fit the index
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documets)

In [19]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [12]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}
    
    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )

    return results

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [20]:
def llm(prompt):

    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
)
    return response.choices[0].message.content.strip()

In [26]:
query  = "how do i run kafka"

def rag(query):
    search_results = search(query)

    prompt = build_prompt(query, search_results)

    answer = llm(prompt)
    return answer

In [27]:
rag("hello there")

'Hello, how can I assist you with the course today? Do you have a specific question or need help with one of the modules?'